## Embeddings

No exemplo anterior, trabalhamos com vetores de bag-of-words de alta dimensão com comprimento `vocab_size`, e estávamos convertendo explicitamente de vetores de representação posicional de baixa dimensão para uma representação esparsa de uma única posição ativa (one-hot). Essa representação one-hot não é eficiente em termos de memória e, além disso, cada palavra é tratada de forma independente, ou seja, vetores codificados em one-hot não expressam nenhuma similaridade semântica entre as palavras.

Nesta unidade, continuaremos explorando o conjunto de dados **News AG**. Para começar, vamos carregar os dados e obter algumas definições do notebook anterior.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## O que é embedding?

A ideia de **embedding** é representar palavras por vetores densos de menor dimensão, que de alguma forma refletem o significado semântico de uma palavra. Mais adiante, discutiremos como construir embeddings de palavras significativos, mas, por enquanto, vamos apenas pensar em embeddings como uma forma de reduzir a dimensionalidade de um vetor de palavras.

Assim, a camada de embedding receberia uma palavra como entrada e produziria um vetor de saída com o `embedding_size` especificado. De certa forma, é muito semelhante à camada `Linear`, mas, em vez de receber um vetor codificado em one-hot, ela será capaz de receber um número que representa a palavra como entrada.

Ao usar a camada de embedding como a primeira camada em nossa rede, podemos mudar do modelo bag-of-words para o modelo **embedding bag**, onde primeiro convertemos cada palavra em nosso texto no embedding correspondente e, em seguida, calculamos alguma função de agregação sobre todos esses embeddings, como `sum`, `average` ou `max`.

![Imagem mostrando um classificador de embedding para cinco palavras em sequência.](../../../../../translated_images/pt-BR/embedding-classifier-example.b77f021a7ee67eee.webp)

Nossa rede neural classificadora começará com uma camada de embedding, seguida por uma camada de agregação e, por fim, um classificador linear no topo:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Lidando com tamanhos variáveis de sequência

Como resultado dessa arquitetura, os minibatches para nossa rede precisariam ser criados de uma maneira específica. Na unidade anterior, ao usar bag-of-words, todos os tensores BoW em um minibatch tinham tamanho igual a `vocab_size`, independentemente do comprimento real da sequência de texto. Quando passamos a usar embeddings de palavras, acabamos lidando com um número variável de palavras em cada amostra de texto, e ao combinar essas amostras em minibatches, precisaríamos aplicar algum preenchimento (padding).

Isso pode ser feito utilizando a mesma técnica de fornecer a função `collate_fn` para a fonte de dados:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Treinando o classificador de embeddings

Agora que definimos um dataloader adequado, podemos treinar o modelo usando a função de treinamento que definimos na unidade anterior:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Nota**: Estamos treinando apenas 25 mil registros aqui (menos de uma época completa) por questão de tempo, mas você pode continuar treinando, escrever uma função para treinar por várias épocas e experimentar com o parâmetro de taxa de aprendizado para alcançar maior precisão. Você deve ser capaz de atingir uma precisão de cerca de 90%.


### Camada EmbeddingBag e Representação de Sequências de Comprimento Variável

Na arquitetura anterior, precisávamos preencher todas as sequências para que tivessem o mesmo comprimento, a fim de ajustá-las em um minibatch. Essa não é a maneira mais eficiente de representar sequências de comprimento variável - outra abordagem seria usar um vetor de **offset**, que armazenaria os deslocamentos de todas as sequências em um único vetor grande.

![Imagem mostrando uma representação de sequência com offset](../../../../../translated_images/pt-BR/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Na imagem acima, mostramos uma sequência de caracteres, mas em nosso exemplo estamos trabalhando com sequências de palavras. No entanto, o princípio geral de representar sequências com um vetor de offset permanece o mesmo.

Para trabalhar com a representação de offset, usamos a camada [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Ela é semelhante à `Embedding`, mas recebe um vetor de conteúdo e um vetor de offset como entrada, e também inclui uma camada de agregação, que pode ser `mean`, `sum` ou `max`.

Aqui está uma rede modificada que utiliza `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Para preparar o conjunto de dados para treinamento, precisamos fornecer uma função de conversão que irá preparar o vetor de deslocamento:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Note que, diferentemente de todos os exemplos anteriores, nossa rede agora aceita dois parâmetros: vetor de dados e vetor de deslocamento, que possuem tamanhos diferentes. Da mesma forma, nosso carregador de dados também nos fornece 3 valores em vez de 2: tanto os vetores de texto quanto os vetores de deslocamento são fornecidos como características. Portanto, precisamos ajustar ligeiramente nossa função de treinamento para lidar com isso:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Embeddings Semânticos: Word2Vec

No nosso exemplo anterior, a camada de embedding do modelo aprendeu a mapear palavras para representações vetoriais, porém, essa representação não tinha muito significado semântico. Seria interessante aprender uma representação vetorial em que palavras semelhantes ou sinônimos corresponderiam a vetores próximos entre si em termos de alguma distância vetorial (por exemplo, distância euclidiana).

Para isso, precisamos pré-treinar nosso modelo de embedding em uma grande coleção de textos de uma maneira específica. Uma das primeiras abordagens para treinar embeddings semânticos é chamada de [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ela se baseia em duas arquiteturas principais que são usadas para produzir uma representação distribuída de palavras:

 - **Continuous bag-of-words** (CBoW) — nesta arquitetura, treinamos o modelo para prever uma palavra a partir do contexto ao redor. Dado o ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, o objetivo do modelo é prever $W_0$ a partir de $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** é o oposto do CBoW. O modelo usa a janela de palavras de contexto ao redor para prever a palavra atual.

CBoW é mais rápido, enquanto skip-gram é mais lento, mas faz um trabalho melhor ao representar palavras menos frequentes.

![Imagem mostrando os algoritmos CBoW e Skip-Gram para converter palavras em vetores.](../../../../../translated_images/pt-BR/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Para experimentar com embeddings word2vec pré-treinados no conjunto de dados Google News, podemos usar a biblioteca **gensim**. Abaixo, encontramos as palavras mais semelhantes a 'neural'.

> **Nota:** Quando você cria vetores de palavras pela primeira vez, baixá-los pode levar algum tempo!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Podemos também calcular embeddings de vetor a partir da palavra, para serem usados no treinamento do modelo de classificação (mostramos apenas os primeiros 20 componentes do vetor para maior clareza):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

A grande vantagem das incorporações semânticas é que você pode manipular a codificação vetorial para alterar a semântica. Por exemplo, podemos pedir para encontrar uma palavra cuja representação vetorial seja o mais próxima possível das palavras *rei* e *mulher*, e o mais distante possível da palavra *homem*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Ambos CBoW e Skip-Grams são embeddings "previsores", pois consideram apenas os contextos locais. Word2Vec não aproveita o contexto global.

**FastText** expande o Word2Vec ao aprender representações vetoriais para cada palavra e os n-gramas de caracteres encontrados dentro de cada palavra. Os valores das representações são então calculados como uma média em um único vetor a cada etapa de treinamento. Embora isso adicione muita computação adicional ao pré-treinamento, permite que os embeddings de palavras codifiquem informações de subpalavras.

Outro método, **GloVe**, utiliza a ideia de matriz de coocorrência e emprega métodos neurais para decompor a matriz de coocorrência em vetores de palavras mais expressivos e não lineares.

Você pode experimentar o exemplo alterando os embeddings para FastText e GloVe, já que o gensim suporta vários modelos diferentes de embeddings de palavras.


## Usando Embeddings Pré-Treinados no PyTorch

Podemos modificar o exemplo acima para pré-preencher a matriz em nossa camada de embedding com embeddings semânticos, como Word2Vec. Precisamos levar em conta que os vocabulários do embedding pré-treinado e do nosso corpus de texto provavelmente não irão coincidir, então inicializaremos os pesos para as palavras ausentes com valores aleatórios:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Agora vamos treinar nosso modelo. Observe que o tempo necessário para treinar o modelo é significativamente maior do que no exemplo anterior, devido ao tamanho maior da camada de embedding e, consequentemente, ao número muito maior de parâmetros. Além disso, por causa disso, podemos precisar treinar nosso modelo em mais exemplos se quisermos evitar overfitting.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

No nosso caso, não observamos um grande aumento na precisão, o que provavelmente se deve a vocabulários bastante diferentes.  
Para superar o problema de vocabulários distintos, podemos usar uma das seguintes soluções:  
* Re-treinar o modelo word2vec com o nosso vocabulário  
* Carregar nosso conjunto de dados com o vocabulário do modelo word2vec pré-treinado. O vocabulário usado para carregar o conjunto de dados pode ser especificado durante o carregamento.  

A última abordagem parece mais fácil, especialmente porque o framework `torchtext` do PyTorch contém suporte integrado para embeddings. Podemos, por exemplo, instanciar um vocabulário baseado em GloVe da seguinte maneira:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


O vocabulário carregado possui as seguintes operações básicas:
* O dicionário `vocab.stoi` nos permite converter uma palavra em seu índice no dicionário.
* `vocab.itos` faz o oposto - converte um número em uma palavra.
* `vocab.vectors` é o array de vetores de embeddings, então, para obter o embedding de uma palavra `s`, precisamos usar `vocab.vectors[vocab.stoi[s]]`.

Aqui está um exemplo de manipulação de embeddings para demonstrar a equação **kind-man+woman = queen** (tive que ajustar um pouco o coeficiente para funcionar):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Para treinar o classificador usando esses embeddings, primeiro precisamos codificar nosso conjunto de dados usando o vocabulário GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Como vimos acima, todas as incorporações de vetores são armazenadas na matriz `vocab.vectors`. Isso torna super fácil carregar esses pesos na camada de pesos de incorporação usando uma cópia simples:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Um dos motivos pelos quais não estamos vendo um aumento significativo na precisão é devido ao fato de que algumas palavras do nosso conjunto de dados estão ausentes no vocabulário pré-treinado do GloVe e, portanto, são essencialmente ignoradas. Para superar esse fato, podemos treinar nossas próprias embeddings em nosso conjunto de dados.


## Embeddings Contextuais

Uma limitação importante das representações tradicionais de embeddings pré-treinados, como o Word2Vec, é o problema de desambiguação de sentidos das palavras. Embora os embeddings pré-treinados consigam capturar parte do significado das palavras no contexto, todos os possíveis significados de uma palavra são codificados no mesmo embedding. Isso pode causar problemas em modelos subsequentes, já que muitas palavras, como a palavra 'play', possuem significados diferentes dependendo do contexto em que são usadas.

Por exemplo, a palavra 'play' nas duas frases abaixo tem significados bastante diferentes:
- Eu fui a uma **peça** no teatro.
- John quer **brincar** com seus amigos.

Os embeddings pré-treinados acima representam ambos os significados da palavra 'play' no mesmo embedding. Para superar essa limitação, precisamos construir embeddings baseados no **modelo de linguagem**, que é treinado em um grande corpus de texto e *sabe* como as palavras podem ser combinadas em diferentes contextos. Discutir embeddings contextuais está fora do escopo deste tutorial, mas voltaremos a esse tema ao falar sobre modelos de linguagem na próxima unidade.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
